# San Antonio Hotel Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
comgas_cost = 0.60
comelec_cost = 0.0806

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.53

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI and cost/sf

sf = 189.6

In [6]:
# Lighting energy reduction, assumption that LEDs are being used.

lighting_redux = .3

---

## Combine CSV Files in Directory Commercial Electric Heating

In [7]:
# Create variable for files in directory
files = [f for f in os.listdir("data/sat/") if f.endswith(".csv")]

# files

In [8]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/sat/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 394200  rows, 44 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,A1Eclear,E,Clear,01/01 01:00:00,14.875,0.975,35493.11829,0.0,0.0,0.0,...,6.263576,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,53979.40333
1,A1Eclear,E,Clear,01/01 02:00:00,12.375,0.975,35493.11829,0.0,0.0,0.0,...,6.051370,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
2,A1Eclear,E,Clear,01/01 03:00:00,12.000,0.975,15016.31927,0.0,0.0,0.0,...,5.794059,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
3,A1Eclear,E,Clear,01/01 04:00:00,12.000,0.975,15016.31927,0.0,0.0,0.0,...,5.632373,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
4,A1Eclear,E,Clear,01/01 05:00:00,12.625,0.975,15016.31927,0.0,0.0,0.0,...,5.571512,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,C3Wlowe,W,Lowe,12/31 20:00:00,15.125,0.975,136511.99340,0.0,0.0,0.0,...,5.024463,1.0,1.0,12.8,21.768,0.0,0.000000,0,188544.6314,22933.84749
8756,C3Wlowe,W,Lowe,12/31 21:00:00,13.375,0.975,136511.99340,0.0,0.0,0.0,...,5.012151,1.0,1.0,12.8,21.768,0.0,0.000000,0,188544.6314,0.00000
8757,C3Wlowe,W,Lowe,12/31 22:00:00,11.750,0.750,136511.99340,0.0,0.0,0.0,...,5.015886,1.0,1.0,12.8,21.768,0.0,0.000000,0,188544.6314,0.00000
8758,C3Wlowe,W,Lowe,12/31 23:00:00,11.000,0.750,116035.19440,0.0,0.0,0.0,...,5.241458,1.0,1.0,12.8,21.768,0.0,7449.151542,0,188544.6314,0.00000


In [9]:
# list(all_ComElec.columns.values)

In [10]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [11]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

-----

## Get the Wanted Data (columns)

In [12]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ComElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Add EUI (site) column


In [13]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [14]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [15]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [16]:
# Add energy totals
totalComEnergy = get_annual_ComElec["FanEnergy(kBtu)"] + get_annual_ComElec["HeatingElecEnergy(kBtu)"] + get_annual_ComElec["LightingEnergy(kBtu)"] + get_annual_ComElec["CoolingEnergy(kBtu)"]
        
get_annual_ComElec["AnnualEnergy(kBtu)"] = totalComEnergy

# get_annual_ComElec

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [17]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [18]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingGasEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ComElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [19]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingGasEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier
Add cost multiplier, variables defined above

In [20]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [21]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = ((get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [22]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

In [23]:
# Add cost totals
totalComCost = get_annual_costs_ComElec["FanCost($)"] + get_annual_costs_ComElec["HeatingElecCost($)"] + get_annual_costs_ComElec["HeatingGasCost($)"]  + get_annual_costs_ComElec["CoolingCost($)"] + get_annual_costs_ComElec["LightingCost($)"]
        
get_annual_costs_ComElec["AnnualCost($)"] = totalComCost

# get_annual_costs_ComElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [24]:
# Get data per sf ($/sf)
get_annual_costs_ComElec["FanCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"]) / sf
get_annual_costs_ComElec["HeatingElecCost($/sf)"] = (get_annual_costs_ComElec["HeatingElecCost($)"]) / sf
get_annual_costs_ComElec["HeatingGasCost($/sf)"] = (get_annual_costs_ComElec["HeatingGasCost($)"]) / sf
get_annual_costs_ComElec["CoolingCost($/sf)"] = (get_annual_costs_ComElec["CoolingCost($)"]) / sf
get_annual_costs_ComElec["LightingCost($/sf)"] = (get_annual_costs_ComElec["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [25]:
# Establish costs ($/sf)
get_annual_costs_ComElec["AnnualCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"] +
                                    get_annual_costs_ComElec["HeatingElecCost($)"] +
                                    get_annual_costs_ComElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ComElec["CoolingCost($)"] +  
                                    get_annual_costs_ComElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [26]:
# # Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

# allAnnualDataComElec

In [27]:
# list(allAnnualDataComElec.columns.values)

---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [28]:
# # Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataComElec

In [29]:
# Rename columns to indicate baseline to prepare for merge\n"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={"Scenario":"Scenario",
 "Date_Time":"Date_Time",
 "Elevation":"Elevation",
 "GlazingSystem":"B_GlazingSystem",
 "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
 "HeatingElecEnergy(kBtu)":"B_HeatingElecEnergy(kBtu)",
 "HeatingGasEnergy(kBtu)":"B_HeatingGasEnergy(kBtu)",
 "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
 "LightingEnergy(kBtu)":"B_LightingEnergy(kBtu)",
 "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
 "FanEnergy(kBtu/sf)":"B_FanEnergy(kBtu/sf)",
 "HeatingElecEnergy(kBtu/sf)":"B_HeatingElecEnergy(kBtu/sf)",
 "HeatingGasEnergy(kBtu/sf)":"B_HeatingGasEnergy(kBtu/sf)",
 "CoolingEnergy(kBtu/sf)":"B_CoolingEnergy(kBtu/sf)",
 "LightingEnergy(kBtu/sf)":"B_LightingEnergy(kBtu/sf)",
 "EUI(kBtu/sf)":"B_EUI(kBtu/sf)",
 "CO2_Electric(lb/sf)":"B_CO2_Electric(lb/sf)",
 "FanCost($)":"B_FanCost($)",
 "HeatingElecCost($)":"B_HeatingElecCost($)",
 "HeatingGasCost($)":"B_HeatingGasCost($)",
 "CoolingCost($)":"B_CoolingCost($)",
 "LightingCost($)":"B_LightingCost($)",
 "AnnualCost($)":"B_AnnualCost($)",
 "FanCost($/sf)":"B_FanCost($/sf)",
 "HeatingElecCost($/sf)":"B_HeatingElecCost($/sf)",
 "HeatingGasCost($/sf)":"B_HeatingGasCost($/sf)",
 "CoolingCost($/sf)":"B_CoolingCost($/sf)",
 "LightingCost($/sf)":"B_LightingCost($/sf)",
 "AnnualCost($/sf)":"B_AnnualCost($/sf)"
                            })

# allBaselineAnnualDataComElec

In [30]:
# Merge the dataframes to include the baseline data.
AnnualWithBaselineCom = pd.merge(allAnnualDataComElec, allBaselineAnnualDataComElec, on=["Date_Time", "Elevation", "Scenario"], how="left")

# AnnualWithBaselineCom

In [31]:
# list(AnnualWithBaselineCom.columns.values)

In [32]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaselineCom["deltaEnergy(kBtu)"] = AnnualWithBaselineCom["AnnualEnergy(kBtu)"] - AnnualWithBaselineCom["B_AnnualEnergy(kBtu)"]
AnnualWithBaselineCom["deltaEUI(kBtu/sf)"] = AnnualWithBaselineCom["EUI(kBtu/sf)"] - AnnualWithBaselineCom["B_EUI(kBtu/sf)"]
AnnualWithBaselineCom["deltaCO2Elec(lb/sf)"] = AnnualWithBaselineCom["CO2_Electric(lb/sf)"] - AnnualWithBaselineCom["B_CO2_Electric(lb/sf)"]
AnnualWithBaselineCom["deltaCost($)"] = AnnualWithBaselineCom["AnnualCost($)"] - AnnualWithBaselineCom["B_AnnualCost($)"]
AnnualWithBaselineCom["deltaCost($/sf)"] = AnnualWithBaselineCom["AnnualCost($/sf)"] - AnnualWithBaselineCom["B_AnnualCost($/sf)"]

# AnnualWithBaselineCom

In [33]:
# Convert NANs to 0
AnnualWithBaselineCom =AnnualWithBaselineCom.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario
Export to file

In [34]:
# Group by scenario
annualDataCom = AnnualWithBaselineCom.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

annualDataCom.sum().round(2)

,,,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),...,B_HeatingElecCost($/sf),B_HeatingGasCost($/sf),B_CoolingCost($/sf),B_LightingCost($/sf),B_AnnualCost($/sf),deltaEnergy(kBtu),deltaEUI(kBtu/sf),deltaCO2Elec(lb/sf),deltaCost($),deltaCost($/sf)
Scenario,Elevation,GlazingSystem,,,,,,,,,,,,,,,,,,,,,
A1Eclear,E,Clear,1381.36,67.95,0.0,4096.20,140.14,5685.64,7.29,0.36,0.0,21.60,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Eexist,E,Exist,1349.58,380.67,0.0,4070.76,140.14,5941.14,7.12,2.01,0.0,21.47,...,0.01,0.0,0.51,0.74,1.43,0.0,0.0,0.0,0.0,0.0
A1Elowe,E,Lowe,1198.60,145.44,0.0,3264.34,140.14,4748.52,6.32,0.77,0.0,17.22,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Nclear,N,Clear,1332.99,186.25,0.0,3151.07,140.14,4810.45,7.03,0.98,0.0,16.62,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Nexist,N,Exist,1288.73,741.80,0.0,3157.01,140.14,5327.68,6.80,3.91,0.0,16.65,...,0.02,0.0,0.39,0.74,1.32,0.0,0.0,0.0,0.0,0.0
A1Nlowe,N,Lowe,1167.58,308.68,0.0,2666.33,140.14,4282.73,6.16,1.63,0.0,14.06,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Wclear,W,Clear,2030.88,29.68,0.0,5065.73,140.14,7266.43,10.71,0.16,0.0,26.72,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Wexist,W,Exist,2116.02,251.53,0.0,5269.30,140.14,7776.97,11.16,1.33,0.0,27.79,...,0.01,0.0,0.66,0.74,1.67,0.0,0.0,0.0,0.0,0.0
A1Wlowe,W,Lowe,1628.93,87.15,0.0,3855.52,140.14,5711.74,8.59,0.46,0.0,20.33,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [35]:
# # Group by scenario
# annualDataCom = AnnualWithBaselineCom.groupby(["Scenario"], as_index=True)

# annualDataCom.sum().round(2)

In [36]:
annualDataComFinal = annualDataCom.sum().round(2)

annualDataComFinal

,,,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),...,B_HeatingElecCost($/sf),B_HeatingGasCost($/sf),B_CoolingCost($/sf),B_LightingCost($/sf),B_AnnualCost($/sf),deltaEnergy(kBtu),deltaEUI(kBtu/sf),deltaCO2Elec(lb/sf),deltaCost($),deltaCost($/sf)
Scenario,Elevation,GlazingSystem,,,,,,,,,,,,,,,,,,,,,
A1Eclear,E,Clear,1381.36,67.95,0.0,4096.20,140.14,5685.64,7.29,0.36,0.0,21.60,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Eexist,E,Exist,1349.58,380.67,0.0,4070.76,140.14,5941.14,7.12,2.01,0.0,21.47,...,0.01,0.0,0.51,0.74,1.43,0.0,0.0,0.0,0.0,0.0
A1Elowe,E,Lowe,1198.60,145.44,0.0,3264.34,140.14,4748.52,6.32,0.77,0.0,17.22,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Nclear,N,Clear,1332.99,186.25,0.0,3151.07,140.14,4810.45,7.03,0.98,0.0,16.62,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Nexist,N,Exist,1288.73,741.80,0.0,3157.01,140.14,5327.68,6.80,3.91,0.0,16.65,...,0.02,0.0,0.39,0.74,1.32,0.0,0.0,0.0,0.0,0.0
A1Nlowe,N,Lowe,1167.58,308.68,0.0,2666.33,140.14,4282.73,6.16,1.63,0.0,14.06,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Wclear,W,Clear,2030.88,29.68,0.0,5065.73,140.14,7266.43,10.71,0.16,0.0,26.72,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
A1Wexist,W,Exist,2116.02,251.53,0.0,5269.30,140.14,7776.97,11.16,1.33,0.0,27.79,...,0.01,0.0,0.66,0.74,1.67,0.0,0.0,0.0,0.0,0.0
A1Wlowe,W,Lowe,1628.93,87.15,0.0,3855.52,140.14,5711.74,8.59,0.46,0.0,20.33,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [37]:
# Export grouped file to csv
annualDataComFinal.to_csv("Scraper_Output/sat_annual_com.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [38]:
# Start with renaming df
get_peak_ComElec = get_annual_ComElec
#get_peak_ComElec

In [39]:
# Add up heat, cool and fans for a total column
get_peak_ComElec["TotalEnergy(kBtu)"] = (get_peak_ComElec["FanEnergy(kBtu)"] +
                                    get_peak_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_peak_ComElec["HeatingGasEnergy(kBtu)"] +
                                    get_peak_ComElec["LightingEnergy(kBtu)"] +
                                    get_peak_ComElec["CoolingEnergy(kBtu)"])

#get_peak_ComElec

In [40]:
# Get scenario and find the max
A1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Eexist"]
A1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Eclear"]
A1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Elowe"]
A1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nexist"]
A1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nclear"]
A1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nlowe"]
A1Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wexist"]
A1Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wclear"]
A1Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wlowe"]
A2Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Sexist"]
A2Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Sclear"]
A2Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Slowe"]
B1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Eexist"]
B1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Eclear"]
B1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Elowe"]
B1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nexist"]
B1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nclear"]
B1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nlowe"]
B1Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wexist"]
B1Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wclear"]
B1Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wlowe"]
B2Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Eexist"]
B2Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Elear"]
B2Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Elowe"]
B3Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Sexist"]
B3Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Sclear"]
B3Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Slowe"]
B3Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wexist"]
B3Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wclear"]
B3Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wlowe"]
C1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Eexist"]
C1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Eclear"]
C1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Elowe"]
C1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nexist"]
C1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nclear"]
C1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nlowe"]
C1Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Sexist"]
C1Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Sclear"]
C1Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Slowe"]
C2Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wexist"]
C2Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wclear"]
C2Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wlowe"]
C3Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wexist"]
C3Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wclear"]
C3Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wlowe"]


In [41]:
A1EexistPeak = (A1Eexist[A1Eexist["TotalEnergy(kBtu)"] == A1Eexist["TotalEnergy(kBtu)"].max()])
A1EclearPeak = (A1Eclear[A1Eclear["TotalEnergy(kBtu)"] == A1Eclear["TotalEnergy(kBtu)"].max()])
A1ElowePeak = (A1Elowe[A1Elowe["TotalEnergy(kBtu)"] == A1Elowe["TotalEnergy(kBtu)"].max()])
A1NexistPeak = (A1Nexist[A1Nexist["TotalEnergy(kBtu)"] == A1Nexist["TotalEnergy(kBtu)"].max()])
A1NclearPeak = (A1Nclear[A1Nclear["TotalEnergy(kBtu)"] == A1Nclear["TotalEnergy(kBtu)"].max()])
A1NlowePeak = (A1Nlowe[A1Nlowe["TotalEnergy(kBtu)"] == A1Nlowe["TotalEnergy(kBtu)"].max()])
A1WexistPeak = (A1Wexist[A1Wexist["TotalEnergy(kBtu)"] == A1Wexist["TotalEnergy(kBtu)"].max()])
A1WclearPeak = (A1Wclear[A1Wclear["TotalEnergy(kBtu)"] == A1Wclear["TotalEnergy(kBtu)"].max()])
A1WlowePeak = (A1Wlowe[A1Wlowe["TotalEnergy(kBtu)"] == A1Wlowe["TotalEnergy(kBtu)"].max()])
A2SexisttPeak = (A2Sexist[A2Sexist["TotalEnergy(kBtu)"] == A2Sexist["TotalEnergy(kBtu)"].max()])
A2SclearPeak = (A2Sclear[A2Sclear["TotalEnergy(kBtu)"] == A2Sclear["TotalEnergy(kBtu)"].max()])
A2SlowePeak = (A2Slowe[A2Slowe["TotalEnergy(kBtu)"] == A2Slowe["TotalEnergy(kBtu)"].max()])
B1EexistPeak = (B1Eexist[B1Eexist["TotalEnergy(kBtu)"] == B1Eexist["TotalEnergy(kBtu)"].max()])
B1EclearPeak = (B1Eclear[B1Eclear["TotalEnergy(kBtu)"] == B1Eclear["TotalEnergy(kBtu)"].max()])
B1ElowePeak = (B1Elowe[B1Elowe["TotalEnergy(kBtu)"] == B1Elowe["TotalEnergy(kBtu)"].max()])
B1NexistPeak = (B1Nexist[B1Nexist["TotalEnergy(kBtu)"] == B1Nexist["TotalEnergy(kBtu)"].max()])
B1NclearPeak = (B1Nclear[B1Nclear["TotalEnergy(kBtu)"] == B1Nclear["TotalEnergy(kBtu)"].max()])
B1NlowePeak = (B1Nlowe[B1Nlowe["TotalEnergy(kBtu)"] == B1Nlowe["TotalEnergy(kBtu)"].max()])
B1WexistPeak = (B1Wexist[B1Wexist["TotalEnergy(kBtu)"] == B1Wexist["TotalEnergy(kBtu)"].max()])
B1WclearPeak = (B1Wclear[B1Wclear["TotalEnergy(kBtu)"] == B1Wclear["TotalEnergy(kBtu)"].max()])
B1WlowePeak = (B1Wlowe[B1Wlowe["TotalEnergy(kBtu)"] == B1Wlowe["TotalEnergy(kBtu)"].max()])
B2EexistPeak = (B2Eexist[B2Eexist["TotalEnergy(kBtu)"] == B2Eexist["TotalEnergy(kBtu)"].max()])
B2EclearPeak = (B2Eclear[B2Eclear["TotalEnergy(kBtu)"] == B2Eclear["TotalEnergy(kBtu)"].max()])
B2ElowePeak = (B2Elowe[B2Elowe["TotalEnergy(kBtu)"] == B2Elowe["TotalEnergy(kBtu)"].max()])
B3SexistPeak = (B3Sexist[B3Sexist["TotalEnergy(kBtu)"] == B3Sexist["TotalEnergy(kBtu)"].max()])
B3SclearPeak = (B3Sclear[B3Sclear["TotalEnergy(kBtu)"] == B3Sclear["TotalEnergy(kBtu)"].max()])
B3SlowePeak = (B3Slowe[B3Slowe["TotalEnergy(kBtu)"] == B3Slowe["TotalEnergy(kBtu)"].max()])
B3WexistPeak = (B3Wexist[B3Wexist["TotalEnergy(kBtu)"] == B3Wexist["TotalEnergy(kBtu)"].max()])
B3WclearPeak = (B3Wclear[B3Wclear["TotalEnergy(kBtu)"] == B3Wclear["TotalEnergy(kBtu)"].max()])
B3WlowePeak = (B3Wlowe[B3Wlowe["TotalEnergy(kBtu)"] == B3Wlowe["TotalEnergy(kBtu)"].max()])
C1EexistPeak = (C1Eexist[C1Eexist["TotalEnergy(kBtu)"] == C1Eexist["TotalEnergy(kBtu)"].max()])
C1EclearPeak = (C1Eclear[C1Eclear["TotalEnergy(kBtu)"] == C1Eclear["TotalEnergy(kBtu)"].max()])
C1ElowePeak = (C1Elowe[C1Elowe["TotalEnergy(kBtu)"] == C1Elowe["TotalEnergy(kBtu)"].max()])
C1NexistPeak = (C1Nexist[C1Nexist["TotalEnergy(kBtu)"] == C1Nexist["TotalEnergy(kBtu)"].max()])
C1NclearPeak = (C1Nclear[C1Nclear["TotalEnergy(kBtu)"] == C1Nclear["TotalEnergy(kBtu)"].max()])
C1NlowePeak = (C1Nlowe[C1Nlowe["TotalEnergy(kBtu)"] == C1Nlowe["TotalEnergy(kBtu)"].max()])
C1SexistPeak = (C1Sexist[C1Sexist["TotalEnergy(kBtu)"] == C1Sexist["TotalEnergy(kBtu)"].max()])
C1SclearPeak = (C1Sclear[C1Sclear["TotalEnergy(kBtu)"] == C1Sclear["TotalEnergy(kBtu)"].max()])
C1SlowePeak = (C1Slowe[C1Slowe["TotalEnergy(kBtu)"] == C1Slowe["TotalEnergy(kBtu)"].max()])
C2WexistPeak = (C2Wexist[C2Wexist["TotalEnergy(kBtu)"] == C2Wexist["TotalEnergy(kBtu)"].max()])
C2WclearPeak = (C2Wclear[C2Wclear["TotalEnergy(kBtu)"] == C2Wclear["TotalEnergy(kBtu)"].max()])
C2WlowePeak = (C2Wlowe[C2Wlowe["TotalEnergy(kBtu)"] == C2Wlowe["TotalEnergy(kBtu)"].max()])
C3WexistPeak = (C3Wexist[C3Wexist["TotalEnergy(kBtu)"] == C3Wexist["TotalEnergy(kBtu)"].max()])
C3WclearPeak = (C3Wclear[C3Wclear["TotalEnergy(kBtu)"] == C3Wclear["TotalEnergy(kBtu)"].max()])
C3WlowePeak = (C3Wlowe[C3Wlowe["TotalEnergy(kBtu)"] == C3Wlowe["TotalEnergy(kBtu)"].max()])


In [42]:
PeakDemandElec = A1EexistPeak.append([A1EclearPeak,
A1ElowePeak,
A1NexistPeak,
A1NclearPeak,
A1NlowePeak,
A1WexistPeak,
A1WclearPeak,
A1WlowePeak,
A2SexisttPeak,
A2SclearPeak,
A2SlowePeak,
B1EexistPeak,
B1EclearPeak,
B1ElowePeak,
B1NexistPeak,
B1NclearPeak,
B1NlowePeak,
B1WexistPeak,
B1WclearPeak,
B1WlowePeak,
B2EexistPeak,
B2EclearPeak,
B2ElowePeak,
B3SexistPeak,
B3SclearPeak,
B3SlowePeak,
B3WexistPeak,
B3WclearPeak,
B3WlowePeak,
C1EexistPeak,
C1EclearPeak,
C1ElowePeak,
C1NexistPeak,
C1NclearPeak,
C1NlowePeak,
C1SexistPeak,
C1SclearPeak,
C1SlowePeak,
C2WexistPeak,
C2WclearPeak,
C2WlowePeak,
C3WexistPeak,
C3WclearPeak,
C3WlowePeak])

# PeakDemandElec

In [43]:
# Export dataframe  to csv
PeakDemandElec.to_csv("Scraper_Output/sat_peak_com.csv", header=True, index=True)